# TF-co-occurences for WP3 - Data (TOBIAS)
### Outline of this notebook:
    1. Constants, Path and Interface Definitions 
    2. Market Basket analysis with tf-comb, for all cluster/celltypes of a tissue
    3. Differential analysis with all market basket analysis (CombObj ´s) of step 2 for the clusters/celltypes of a single tissue. (One DiffCombObj for each tissue)
    4. Analysis for biological questions 
    
The aim is to find transcription-factor-co-occurences for cluster/celltypes of human tissues with the help of the python-library tf-comb. For the data of WP2. The data basis comes from the "cell atlas of chromatin accessibility across 25 adult human tissues"(https://doi.org/10.1101/2021.02.17.431699) 

**Biological question, that we want to answer with this notebook:**

1. Find Transcriptionfactor-co-occurences, which only occure in one (or more) "cluster/celltypes of a tissue".
    Maybe we can identify a cluster through this co-occurences.

**How to use this notbook:**
1. Please adapted the paths in Constants, Path and interface defintions for your approach
2. Please make sure you have installed the kernel as it is described in the ReadMe
3. Check if the WP3 Data structure is correctly provided (ReadMe).
4. Execute each notebook window from top to bottom one after another.

## 1. Constants, Path and Interface Definitions:

In [4]:
from tfcomb import CombObj, DiffCombObj, utils
import os
import pathlib
import pandas as pd
import numpy as np

'''
Constants for this script.

This window contains all paths and constants, which are later used for this juypter notbook.

Please adapt paths or constants, if use other files or folders. 
For example adapted the genome path, if you use another genome. 
'''

# Path to genome fasta file. Is used for the market basket analysis of tfcomb.
genome_path="/mnt/workspace_stud/allstud/homo_sapiens.104.mainChr.fa"

# Path to the jaspar file (contains transcription factors (TF) binding profiles
# as position frequency matrices (PFMs)). Is used for the market basket analysis of tfcomb
main_jaspar_file="../testdaten/JASPAR2020_CORE_vertebrates.meme"  

# Path where results of this notebook will be written to (eg. TF_COMB objects, .pkl).
result_path="./results/wp3/"

# Path to folder (Interface folder) of wp3, where the clusters of each tissue can be found.
path_to_tissues="/mnt/workspace_stud/allstud/wp3/new_tissiues/"

# identifier for the folder names, we need 
#cluster_folder_identifier="cluster"
bindetect_path_snippet = "/output/TFBS/"
conditions_path_snippet = "/clusterBams/"

# Paths in the result folder:
# Path to folder, where the resulting market basket analysis for a cluster/celltype is put 
main_analysis_path=f"{result_path}main/"

# Path to folder, where the differential analysis for a tissue is put 
differential_analysis_path=f"{result_path}diff_analysis/"
# Path to folder, where answers of our results are put to
answers_path=f"{result_path}answers/"

# Path to wp2 cluster for celltype annotation files
path_to_wp2_tissues="/mnt/workspace_stud/stud3/WP2_OUTPUT/FINISHED/"

# # The following lines, initally check if all file/paths are available. 
# If a result folder does not exist it is created automatically
if not os.path.exists(result_path):
     pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)

if not os.path.exists(main_analysis_path):
     pathlib.Path(main_analysis_path).mkdir(parents=True, exist_ok=True)

if not os.path.exists(differential_analysis_path):
     pathlib.Path(differential_analysis_path).mkdir(parents=True, exist_ok=True)

if not os.path.exists(answers_path):
     pathlib.Path(answers_path).mkdir(parents=True, exist_ok=True)

if not os.path.exists(genome_path):
    print(f"ERROR: path {genome_path} does not exist")

if not os.path.exists(main_jaspar_file):
    print(f"ERROR: path {main_jaspar_file} does not exist")

if not os.path.exists(path_to_tissues):
    print(f"ERROR: path {path_to_tissues} does not exist")

### Helper functions for reading-in folders/files:

In [5]:
def get_folder_names_in_folder(rel_folder_path:str):
    ''' 
        Read in the names of the folders in a folder.(rel_folder_path)
        ---
        Parameters:
        
        rel_folder_path: String
            relative Path to the folder location that is read in.
        ---
        Return: a List of Strings (folder names)
    '''

    dirlist = [ item for item in os.listdir(rel_folder_path) if os.path.isdir(os.path.join(rel_folder_path, item))]
    folder_names = []
    for folder in dirlist:
        folder_names.append(folder)
    return folder_names

def read_in_file_names_of_folder(rel_path:str):
    ''' 
        Read in the file names in a folder (rel_path).
        ---
        Parameters:
        
        rel_path: String
            relative Path to the folder location.
        ---
        Return: a List of Strings (file names)
    '''
    return [f for f in os.listdir(rel_path) if os.path.isfile(os.path.join(rel_path, f))]


## 2. Market basket analysis with tf-comb
We do a market basket analysis with tfcomb for each cluster/celltype, which has been culstered by WP3 and comes from the raw single-cell ATAC-Data of the cell atlas project. As a result, we get the transcription-factor-co-occurences for each cluster. The Trancriptionsfactor motif´s come as a position-frequency-matrix from https://jaspar.genereg.net/search?q=&collection=CORE&tax_group=vertebrates. The corresponding genome, which is used is **homo_sapiens.104.mainChr.fa** .

Approach:
1. Read-in tissue folders of WP3
2. For each tissue: read-in single the TOBIAS output files of WP3  (Content= open-chromatin regions) files for each cluster/celltype
3. Do market basket analysis for each cluster/celltype
4. Result .pkl CombObj files can be found under **/{result_path}/{main_analysis_path}/{tissue_name}/{cluster_name}.pkl **

In [3]:
def do_market_basket_analyses(tissue: str, bindetect_path: str, condition: str, mb_file_name:str):
    '''
        Does market basket analyses with bindetect output of Tobias (WP3)
        Saves the tfcomb-Object as .pkl file to main_analysis_path.
        ---
        Paramater:
        
        bindetect_path: string
            Path to the output folder of TOBIAS per tissue
        
        tissue: string
            Name of the tissue that is investigated
         
        condition: string
            Original Name of the cluster from WP3 that is investigated. Condition that is used for making the Tobias output by WP3.
            
        mb_file_name: string
            File name for the CombObj.pkl file that is saved
            
    '''
    # Save path initalization, if folder for tissue does not exists, new folder is created.
    save_path = f'{main_analysis_path}{tissue}/'
    if not os.path.exists(save_path):
         pathlib.Path(save_path).mkdir(parents=True, exist_ok=True)
    
    comb = CombObj()
    # Condition is created by WP3.
    comb.TFBS_from_TOBIAS(bindetect_path=bindetect_path, condition=condition, overwrite=False)
    
    print(f'Start market basket analyses for condition: {condition}')
    comb.market_basket(threads=10)
    # if rules of CombObj´s are empty nothing is saved 
    if len(comb.rules) <= 0:
        print(f'Could not find TF-cooccurences for condition: {condition}')
        return
    print(f'Finished market basket analyses for condition: {condition}')
    print(f'Found rules: {len(comb.rules)}')
    # Save tf-comb obj
    comb.to_pickle(f'{save_path}{mb_file_name}.pkl')
    print(f'Saved: {save_path}{mb_file_name}.pkl')
   
    

In [4]:
# Identifing the tissues to read the cluster/celltype from
tissue_names = get_folder_names_in_folder(rel_folder_path=path_to_tissues)
print(tissue_names)

['A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']


### Functions for adding the celltype annotations of wp2 to the corresponding clusters

In [5]:
def get_celltype_annotation_table(celltype_annotation_path:str) -> pd.DataFrame:
    '''
    Read in the the given annotation file(celltype_annotation_path) with pandas read_table function
    ---
    Parameters:
    celltype_annotation_path : string
        Is the path to a file which has the celltype annotations for the cluster numbers.
        e.g. file content:
        6	Stromal cells
        3	Adipocyte progenitor cells
        4	Stromal cells
        7	Fibroblasts
    ---
    Returns a celltype annotation table as a pandas Dataframe.
    '''
    df = pd.DataFrame()
    df = pd.read_table(celltype_annotation_path, header=None, index_col=None)
    return df

def get_celltype_for_cluster(cluster_number: int, celltype_annotation_table: pd.DataFrame):
    '''
    Find celltype annotation for cluster number in the corresponding celltype_annotation_table
    ---
    Parameters:
    cluster_number: integer
        number of the cluster
    
    celltype_annotation_table: pd.DataFrame
        table with the celltype annotations, e.g. Dataframe
         0	1 
       1 6	Stromal cells
       2 3	Adipocyte progenitor cells
       3 4	Stromal cells
       4 7	Fibroblasts
    ---
    Returns the celltype for a cluster_number. (if no celltype kann be found "na" is returned) 
    '''
    # initalize celltype name with na (not available), if no celltype can be found na is returned
    celltype_name = "na"
    try:
        # Try to find the cell-type annotation in the table, if no annotation can be found key error is raised
        celltype_name = celltype_annotation_table[celltype_annotation_table[0] == int(cluster_number)].iat[0,1]
        # replace whitespaces in celltype names, e.g. "Stromal cell" -> "Stromal_cell"
        celltype_name = celltype_name.replace(" ", "_")
    except (KeyError, IndexError) as e:
        print(f"Could not find celltype annotation for cluster: {cluster_number}, set name to na.")
        celltype_name = "na"
   
    return celltype_name

In [6]:
def get_cluster_number_from_bam_file(cluster_name:str):
    '''
    This function extracts a integer number of a string.
    e.g. Cluster names of WP3 look like cluster2.bam. This function extracts the number. 
    ---
    Parameter:
    cluster_name: string,
     cluster name (cluster1 or cluster20)
    ---
    Return integer number of string as String , None if no number is found.
    '''
    integers_in_name = [int(s) for s in cluster_name if s.isdigit()]

    cluster_nr = ""
    for i in integers_in_name:
       cluster_nr = str(cluster_nr) + str(i)
    
    if cluster_nr == "":
        return None
    
    return cluster_nr



    
#Testing the  get_cluster_number_from_bam_file()   function
txt = "cluster10"
print(get_cluster_number_from_bam_file(cluster_name=txt))
txt = "cluster104"
print(get_cluster_number_from_bam_file(cluster_name=txt))
txt = "cluster1"
print(get_cluster_number_from_bam_file(cluster_name=txt))
txt = "clustersdew"
print(get_cluster_number_from_bam_file(cluster_name=txt))


10
104
1
None


In [7]:
# remove this as soon as wp3 has changed the folder structure
#folder_to_filter = ['output','output_data','presentations','testdata']
#tissue_names = list(filter(lambda x: x not in folder_to_filter , tissue_names))

'''
Here the market basket analyses for all clusters in a tissue are executed and saved as .pkl
for further investigations.
'''
print(f"Tissues:{tissue_names}")

cluster_bindetect_paths = []
conditions_path = []
for tissue_name in tissue_names:
    bindetect_path = f"{path_to_tissues}{tissue_name}{bindetect_path_snippet}"
    conditions_path = f"{path_to_tissues}{tissue_name}{conditions_path_snippet}"
    cluster_file_names = read_in_file_names_of_folder(rel_path=conditions_path)
    
    # read in cluster file names of the conditions produced by wp3
    cluster_file_names = list(filter(lambda x: '.bam' in x , cluster_file_names))
    print(cluster_file_names)
    
     # Get the celltype annotation table of wp2. To add celltype for a cluster. e.g. cluster6 = fibroblast
    celltype_table = get_celltype_annotation_table(
        celltype_annotation_path=f'{path_to_wp2_tissues}{tissue_name}/annotation/annotation.txt') # pandas dataframe is returned
    
    for file in cluster_file_names:
        cluster_name = file.split('.bam')[0]
        print(cluster_name)
        # e.g 10_peaks -> [10 , peaks], cluster_number = 10
        cluster_number = get_cluster_number_from_bam_file(cluster_name=cluster_name)
        
        # Error when no cluster number is present. Tries to use the next cluster.
        if cluster_number == None:
            print(f"ERROR: No cluster_number for cluster: {cluster_name}. Continue with next file.")
            continue
        
        # celltype annotierung with wp2 data
        celltype_name = get_celltype_for_cluster(cluster_number=cluster_number,
                                                 celltype_annotation_table=celltype_table)
        print(celltype_name)
        
        try:
            print(cluster_name)
            print(cluster_number)
            print(celltype_name)
            # e.g. JF1O6_body_of_pancreas_c3_fibroblast
            mb_file_name = f"{tissue_name}_c{str(cluster_number)}_{celltype_name}"
            print(f'{bindetect_path}')
            do_market_basket_analyses(tissue=tissue_name,
                                      bindetect_path=bindetect_path,
                                      condition=cluster_name,
                                      mb_file_name=mb_file_name)

        except Exception as err:
            print(err)
            print(f"ERROR: Market basket for cluster:{cluster_name} in tissue {tissue_name}, did not work")
            continue
        
        
print(f"Done Market basket analyses for all tissues")


Tissues:['A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']
['cluster3.bam', 'cluster7.bam', 'cluster8.bam', 'cluster2.bam', 'cluster6.bam', 'cluster1.bam', 'cluster4.bam', 'cluster5.bam']
cluster3
Hepatocytes
cluster3
3
Hepatocytes
/mnt/workspace_stud/allstud/wp3/new_tissiues/A8WNZ_right_lobe_of_liver/output/TFBS/
INFO: Read 4432062 sites (838 unique names) from condition 'cluster3'
Start market basket analyses for condition: cluster3
INFO: Setting up binding sites for counting
INFO: Counting co-occurrences within sites
INFO: Counting co-occurrence within background
INFO: Progress: 10%
INFO: Progress: 20%
INFO: Progress: 30%
INFO: Progress: 40%
INFO: Progress: 50%
INFO: Progress: 60%
INFO: Progress: 70%
INFO: Progress: 80%
INFO: Progress: 90%
INFO: Finished!
INFO: Done finding co-occurrences! Run .market_basket() to estimate significant pairs
INFO: Market basket analysis is done! Results are found in <CombObj>.r

Finished market basket analyses for condition: cluster5
Found rules: 701902
Saved: ./results/wp3/main/A8WNZ_right_lobe_of_liver/A8WNZ_right_lobe_of_liver_c5_Cholangiocytes.pkl
['cluster1.bam', 'cluster3.bam', 'cluster4.bam', 'cluster6.bam', 'cluster2.bam', 'cluster7.bam', 'cluster5.bam']
cluster1
Gastric_chief_cells
cluster1
1
Gastric_chief_cells
/mnt/workspace_stud/allstud/wp3/new_tissiues/ACCQ1_colon_transverse/output/TFBS/
INFO: Read 2168663 sites (838 unique names) from condition 'cluster1'
Start market basket analyses for condition: cluster1
INFO: Setting up binding sites for counting
INFO: Counting co-occurrences within sites
INFO: Counting co-occurrence within background
INFO: Progress: 12%
INFO: Progress: 20%
INFO: Progress: 34%
INFO: Progress: 40%
INFO: Progress: 50%
INFO: Progress: 60%
INFO: Progress: 70%
INFO: Progress: 80%
INFO: Progress: 90%
INFO: Finished!
INFO: Done finding co-occurrences! Run .market_basket() to estimate significant pairs
INFO: Market basket analysis is

INFO: Market basket analysis is done! Results are found in <CombObj>.rules
Finished market basket analyses for condition: cluster2
Found rules: 674920
Saved: ./results/wp3/main/ACCPU_upper_lobe_of_left_lung/ACCPU_upper_lobe_of_left_lung_c2_Adipocytes.pkl
cluster3
Could not find celltype annotation for cluster: 3, set name to na.
na
cluster3
3
na
/mnt/workspace_stud/allstud/wp3/new_tissiues/ACCPU_upper_lobe_of_left_lung/output/TFBS/
INFO: Read 130734 sites (838 unique names) from condition 'cluster3'
Start market basket analyses for condition: cluster3
INFO: Setting up binding sites for counting
INFO: Counting co-occurrences within sites
INFO: Counting co-occurrence within background
INFO: Progress: 10%
INFO: Progress: 20%
INFO: Progress: 32%
INFO: Progress: 42%
INFO: Progress: 52%
INFO: Progress: 64%
INFO: Progress: 70%
INFO: Progress: 82%
INFO: Progress: 90%
INFO: Finished!
INFO: Done finding co-occurrences! Run .market_basket() to estimate significant pairs
INFO: Market basket analys

INFO: Counting co-occurrences within sites
INFO: Counting co-occurrence within background
INFO: Progress: 10%
INFO: Progress: 20%
INFO: Progress: 30%
INFO: Progress: 40%
INFO: Progress: 50%
INFO: Progress: 60%
INFO: Progress: 70%
INFO: Progress: 80%
INFO: Progress: 90%
INFO: Finished!
INFO: Done finding co-occurrences! Run .market_basket() to estimate significant pairs
INFO: Market basket analysis is done! Results are found in <CombObj>.rules
Finished market basket analyses for condition: cluster6
Found rules: 702150
Saved: ./results/wp3/main/A8CPH_esophagus_muscularis_mucosa/A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes.pkl
cluster9
Adipocyte_progenitor_cells
cluster9
9
Adipocyte_progenitor_cells
/mnt/workspace_stud/allstud/wp3/new_tissiues/A8CPH_esophagus_muscularis_mucosa/output/TFBS/
INFO: Read 4920293 sites (838 unique names) from condition 'cluster9'
Start market basket analyses for condition: cluster9
INFO: Setting up binding sites for counting
INFO: Counting co-occurrences w

## 3. Analysis

### Differential Analysis
We use the differential analysis of tfcomb to identifiy the differences of tf-co-occurences between all cluster/celltypes of a tissue. For this we load all market basket analysis (tfcomb-objects) of a tissue (see point 2) into a DiffCombObj. After the differential analysis, we filter the object, so that we hopefully find tf-co-occurences that only occure in a single cluster of that tissue or only in a special celltype.  

- mb = market basket analysis (CombObj of tf comb)

In [8]:
# get the tissue names by folder names of the market basket analysis 
mb_tissues = get_folder_names_in_folder(rel_folder_path=main_analysis_path)
print(mb_tissues)
if '.ipynb_checkpoints' in mb_tissues:
    mb_tissues.remove('.ipynb_checkpoints')
print(mb_tissues)

['.ipynb_checkpoints', 'A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']
['A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']


In [9]:
def do_differential_analysis_for_tissues(tissues=[]):
    '''
        Differential analysis between all clusters/celltypes of a tissue.
        ---
        Paramater:
        
        tissues: array
            tissue names by market basket analysis.
        ---
        DiffCombObj are saved as .pkl to differential_analysis_path
    '''
    for tissue_folder in tissues:
        
        diff_save_path = f"{differential_analysis_path}{tissue_folder}/"
        # Check if  folder for differential_analysis already exists for tissue, if not create new one
        if not os.path.exists(diff_save_path):
             pathlib.Path(diff_save_path).mkdir(parents=True, exist_ok=True)
        
        # get file names of the market basket analysis
        tissue_mb_files = read_in_file_names_of_folder(rel_path=f"{main_analysis_path}{tissue_folder}/")
        
        # holds the combobj´s
        tissue_mbs_to_compare = []
        for file in tissue_mb_files:
            print(file)
            file_name = file.split('.pkl')[0]
            # Load the CombObj (market basket analysis) for each cluster of a tissue
            obj = CombObj().from_pickle(f"{main_analysis_path}{tissue_folder}/{file}")
            obj.set_prefix(file_name)
            tissue_mbs_to_compare.append(obj)
        
        # Create DiffCombObj with all Combobj (market basket analysis) of the clusters in a tissue
        compare_obj = DiffCombObj(tissue_mbs_to_compare, measure="cosine", join="outer", fillna=True)
        # save diffcombj
        compare_obj.to_pickle(f'{diff_save_path}{tissue_folder}.pkl')
        # Normalize the DiffCombObj
        compare_obj.normalize()
        compare_obj.calculate_foldchanges()
        
        # Remove rules which are doubled, e.g. A-B, B-A; (B-A) is removed
        compare_obj.simplify_rules()
        # Save the normalized, foldchange calculated and simplified diff_comb_obj to .pkl
        compare_obj.to_pickle(f'{diff_save_path}{tissue_folder}_normalized.pkl')
        print(f"Done: Diff analysis for tissue {tissue_folder}")
        

In [10]:
# Do the differential analysis for all clusters in a tissue 
do_differential_analysis_for_tissues(tissues=mb_tissues)

A8WNZ_right_lobe_of_liver_c3_Hepatocytes.pkl
A8WNZ_right_lobe_of_liver_c7_Hepatocytes.pkl
A8WNZ_right_lobe_of_liver_c8_Adipocyte_progenitor_cells.pkl
A8WNZ_right_lobe_of_liver_c2_Kupffer_cells.pkl
A8WNZ_right_lobe_of_liver_c6_Hepatocytes.pkl
A8WNZ_right_lobe_of_liver_c1_Adipocytes.pkl
A8WNZ_right_lobe_of_liver_c4_Hepatocytes.pkl
A8WNZ_right_lobe_of_liver_c5_Cholangiocytes.pkl
INFO: Calculating foldchange for contrast: A8WNZ_right_lobe_of_liver_c3_Hepatocytes / A8WNZ_right_lobe_of_liver_c7_Hepatocytes
INFO: Calculating foldchange for contrast: A8WNZ_right_lobe_of_liver_c3_Hepatocytes / A8WNZ_right_lobe_of_liver_c8_Adipocyte_progenitor_cells
INFO: Calculating foldchange for contrast: A8WNZ_right_lobe_of_liver_c3_Hepatocytes / A8WNZ_right_lobe_of_liver_c2_Kupffer_cells
INFO: Calculating foldchange for contrast: A8WNZ_right_lobe_of_liver_c3_Hepatocytes / A8WNZ_right_lobe_of_liver_c6_Hepatocytes
INFO: Calculating foldchange for contrast: A8WNZ_right_lobe_of_liver_c3_Hepatocytes / A8WNZ_righ

A8CPH_esophagus_muscularis_mucosa_c13_Gastric_chief_cells.pkl
A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes.pkl
A8CPH_esophagus_muscularis_mucosa_c9_Adipocyte_progenitor_cells.pkl
A8CPH_esophagus_muscularis_mucosa_c5_Stromal_cells.pkl
A8CPH_esophagus_muscularis_mucosa_c3_Gastric_chief_cells.pkl
A8CPH_esophagus_muscularis_mucosa_c7_Fibroblasts.pkl
A8CPH_esophagus_muscularis_mucosa_c12_na.pkl
A8CPH_esophagus_muscularis_mucosa_c2_Enterocytes.pkl
A8CPH_esophagus_muscularis_mucosa_c4_Enterocytes.pkl
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c10_Stromal_cells / A8CPH_esophagus_muscularis_mucosa_c1_Fibroblasts
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c10_Stromal_cells / A8CPH_esophagus_muscularis_mucosa_c8_Gastric_chief_cells
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c10_Stromal_cells / A8CPH_esophagus_muscularis_mucosa_c11_Fibroblasts
INFO: Calculating foldchange for contrast: A8CPH_e

INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes / A8CPH_esophagus_muscularis_mucosa_c3_Gastric_chief_cells
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes / A8CPH_esophagus_muscularis_mucosa_c7_Fibroblasts
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes / A8CPH_esophagus_muscularis_mucosa_c12_na
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes / A8CPH_esophagus_muscularis_mucosa_c2_Enterocytes
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c6_Enterocytes / A8CPH_esophagus_muscularis_mucosa_c4_Enterocytes
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c9_Adipocyte_progenitor_cells / A8CPH_esophagus_muscularis_mucosa_c5_Stromal_cells
INFO: Calculating foldchange for contrast: A8CPH_esophagus_muscularis_mucosa_c9_Adipocyte_progenitor_cells / A8CPH_esophagus_

# 4. Specific analysis  :
### Biological Question:
 Find Transcriptionfactor-co-occurences, which only occure in one (or more) "cluster/celltype of a tissue".
 - Solution with a hard threshold (a, all cluster) or a week threshold (b, celltypes in a tissue):
       - a) The found TF-pair occures only in a single cluster of a tissue:
            e.g. Cluster1 has TF1-TF1 and the other cluster in the tissue do not have TF1-TF1.
       - b) A TF-pair occures significantly only in some clusters:
            e.g. We have 3 endothelcells cluster in a tissue and this three clusters share a TF-pair
            which does not occure in the other cluster of the tissue. (need celltype annotation)
          
###  Steps:
1. Read in tfcomb-DiffObj´s of the Differential analysis (Point 3)
2. Get the rules of the DiffCombObj
3. Filter the DiffCombObj rules for each cluster in it:
       - e.g. investigate cluster1: get all log2changes col,
         where cluster1 is associated with and his cosine column
       - than filter the rules (tf-pairs) for tf-pairs with a significant change in the log2changes
       - We would like to find TF-pairs that show a significant change in the log2changes in each contrast,
         because this indicates a TF-Pair, which is striking for the investigated cluster in that tissue.
         And a high cosine would be a indication, that the found tf-pair is really 
         a tf-co-occurence in that cluster.
       - To find a TF-pair, which only occures in a single cluster of a tissue,
         we check the log2changes in each col of the significant tf-pairs, if they have the same value,
         the tf-pair only occures in that cluster in the tissue. 

In [11]:
# Get the folder names of the tissues that have a differential analysis
diff_mb_tissues = get_folder_names_in_folder(rel_folder_path=differential_analysis_path)
print(diff_mb_tissues)
if '.ipynb_checkpoints' in diff_mb_tissues:
    diff_mb_tissues.remove('.ipynb_checkpoints')
print(diff_mb_tissues)

['.ipynb_checkpoints', 'A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']
['A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']


In [16]:
def get_significant_rules(df:pd.DataFrame, cosine_col:str, cosine_threshold=0.001, log2fc_threshold_percent=0.10):
    '''
    Filter a rules Dataframe (tf-pairs) by log2foldchange col´s and cosine col´s of a DiffCombObj for significant tf-co-occurences (tf-pairs).
    Only the TF-pairs with significant log2foldchanges per contrast are kept in the returned Dataframe.
    The same filtering is done for the cosine col of the investigated cluster.
    
    For Filtering: for each col (foldchanges/ cosine) the thresholds to get significant values in that col
    are calculated by tfcomb.utils.get_threshold. All rows (tf-pairs), which are not in the threshold 
    (e.g. >95% percentile for log2foldchange or 99.9% percentile) are removed from the datframe (df). 
    Threshold are calculated on the original dataframe (df)
    Filtering/Reduction of the results is done on a copy of the original dataframe.
    
    This is done step by step:
    e.g. 1. thresholds for column "log2change_cluster1/cluster2" are calculated on the original df.
         2. all rows (tf-pairs) of the dataframe,
            where the values of the investigated column (log2change_cluster1/cluster2) are not in the thresholds,
            are removed from the copy of the original dataframe.
        These steps are done for each column.

    ---
    Parameter:
    df: pd.DataFrame
        DiffCombObj Rules
        e.g.:
        index   cosine_cluster1 log2change_cluster1/cluster2 log2change_cluster1/cluster3 log2change_cluster1/cluster4
        TF1-TF2  0.20                        3.4                              1                        4
        TF3-TF4  0.30                       - 0.3                             2                        1
        TFX-TFY  0.15                        8                                3                        -4
        ...
    
    cosine_col: string
        Name of the cluster, which is investigated and name of the cosine col
    
    cosine_threshold: float,
        Threshold value for the cosine col. All values of the cosine col are investigated
        and the threshold value for filtering is calculated. 
        
    log2fc_threshold_percent: float
        Threshold value for the log2foldchanges col. All values of the log2foldchanges cols are investigated
        and the threshold value for filtering is calculated.
    ---
    Return pd.Datframe, Filtered df         

    '''
    # copy of the original dataframe, which will be reduced (remove rows (TF-Pairs),  step by step for each col)
    reduced_df = df.copy(deep=True)
    for col in df.columns:
        # calculate thresholds for log2fc columns or cosine_column
        
        if col == cosine_col:
            # for the cosine column we only want to keep rows with a high value (no negative values),
            # so only the upper threshold is important
            measure_threshold = utils.get_threshold(df[col], "both", percent=cosine_threshold)
            upper_threshold = measure_threshold[1]
            
            # removes tf-pairs (rows), which are smaller than the upper_threshold form the df.
            reduced_df = reduced_df[(reduced_df[col] > upper_threshold)]
            print(upper_threshold)
        else:
            # calculates the thresholds for the log2foldchange cols,
            # positive and negative is possible, so both thresholds are needed.
            measure_threshold = utils.get_threshold(df[col], "both", percent=log2fc_threshold_percent)
            upper_threshold = measure_threshold[1]
            lower_threshold = measure_threshold[0]
             # removes tf-pairs (rows), which are smalleror higher than the thresholds form the df.
            reduced_df = reduced_df[(reduced_df[col] > upper_threshold) | (reduced_df[col] < lower_threshold)]
    # returns only tf-pairs, which are significant        
    return reduced_df

In [20]:
def find_specific_tf_cos_for_cluster(df:pd.DataFrame, cluster_name:str) -> pd.DataFrame:
    '''
        Find tf-co-occurences for a cluster/celltype (cluster_name), that are specific for that cluster
        in the associated tissue.
        
        ---
        Parameters:
            df:pd.DataFrame
                 rules of a DiffCombObj. (Differential analysis)

            cluster_name: string
                name of the cluster/celltype, that is investigated
        ---
        Return pd.Dataframe with tf-pairs, that show a high difference (log2foldchange) to the other clusters in the the tissue
        and are significantly occureing (cosine) in the investigated cluster (cluster_name).   
        
    '''
    # Get only the columns associated with the investigated cluster of the differential analysis,
    # that belongs to the tissue of the cluster.
    # reduce dataframe(df) to relevant columns assocaiated with the cluster
    cluster_cols = list(filter(lambda x: f'{cluster_name}' in x , df.columns))
    # print(cluster_cols)
    relevant_cluster_cols = []
    # Add cosine value for the investigated cluster
    cluster_cosine_col_name = f"{cluster_name}_cosine"
    relevant_cluster_cols.append(cluster_cosine_col_name) 
    for entry in cluster_cols:
        if (f'{cluster_name}/' in entry) or (f'/{cluster_name}_cosine_log2fc' in entry):
            relevant_cluster_cols.append(entry)
   # print(logfc_cluster_cols)
    print(len(relevant_cluster_cols))
    
    # Get only the values (cosine of cluster + log2fc with each contrast of to the cluster) for the cluster to investigate .
    reduced_df = df[relevant_cluster_cols]
    print(f'Initial TF-pairs-Count: {reduced_df.shape}')

   # Filter out rows with 0.00   
    # Count all entries in a row , which do not have a zero(0.00) in it.
    # e.g. 15 cols have 0.00 => val_counts = 0, 10 cols not have a 0.00 => val_counts = 10 
    val_counts = reduced_df[~reduced_df.isin([0])].count(axis=1).sort_values()
    
    # Set threshold 
    selection_threshold = len(relevant_cluster_cols) # e.g. 15, could be varyied
    # Keep all entries, which have more/same values that are higher than the threshold
    tfs_occ = val_counts[val_counts >= selection_threshold].index
    result = reduced_df.loc[tfs_occ]
    print(f'Zero filtered TF-pairs-Count: {result.shape}')
    
    # We would like to find TF-pairs that show a significant change in the log2changes in each contrast, because
    # this indicates a TF-Pair, which is striking for the investigated cluster in that tissue.
    # And a high cosine would be a indication, that the found tf-pair is really a tf-co-occurence in that cluster.
    # Filtering: log2changes and cosine cols to get tf-pairs that have a log2change,
    # that shows a high difference to all other clusters in the tissue and a significant cosine value.
    significants = get_significant_rules(df=result, cosine_col=cluster_cosine_col_name, cosine_threshold=0.001, log2fc_threshold_percent=0.10)
    
    print(f'Cluster: {cluster_name}: {significants.shape} ,tf-pairs with significant log2fc-changes in comparison to all the other clusters in tissue')
    print(f'Done: Find specific tf co-occurences for cluster{cluster_name}.')
    
    return significants

In [21]:
def investigate_differential_analysis_per_tissue(tissues=[]):
    '''
    Investigate the differential analysis of tf-comb for each tissue.
    ---
    Parameters:
    tissues: []
    Tissues that have differential analysis.
    ---
    Exception:
        Catch exception for cluster analysis and continues with analysing the next cluster.
    
    '''
    for tissue in tissues:
        print(f"Start analyse cluster for {tissue}:")
        answer_save_path = f"{answers_path}{tissue}/"
        # Check if  folder for path already exists for tissue, if not create new one
        if not os.path.exists(answer_save_path):
             pathlib.Path(answer_save_path).mkdir(parents=True, exist_ok=True)
        
        # folder errors
        error_path = f"{answers_path}{tissue}/errors/"
        # Check if  folder for path already exists for tissue, if not create new one
        if not os.path.exists(error_path):
             pathlib.Path(error_path).mkdir(parents=True, exist_ok=True)
        
        # load the diffcombobj for the tissue 
        diff_obj = DiffCombObj().from_pickle(f"{differential_analysis_path}{tissue}/{tissue}_normalized.pkl")
        # get the rules (tf-pairs) of the diffcombobj.
        diff_rules = diff_obj.rules
        
        # read in the filenames of the original tfcomobj with the executed market basket analysis. 
        files_main_mb = read_in_file_names_of_folder(rel_path=f"{main_analysis_path}{tissue}")
        
        # Iterate over each cluster.
        for i, file in enumerate(files_main_mb):
            # print(file)
            cluster_name = file.split('.pkl')[0]
            print(f"Start Find specific tf-cos for: {cluster_name}")
            try:
                # res contains a pd.dataframe, that contains tf-pairs, 
                # that have significant log2changes/ and a significant cosine value for the investigated cluster
                # compared to the other clusters in the tissue. This TF-pairs could be interesting for further investigation
                # ,because they show a significant difference to all the other clusters in the tissue.
                res = find_specific_tf_cos_for_cluster(df=diff_rules, cluster_name=cluster_name)

                # save res as .pkl
                res.to_pickle(f"{answer_save_path}{cluster_name}.pkl")
            except Exception as err:
                print(f"ERROR: Could not proceed with analysis for cluster: {cluster_name}")
                print(f"{err}")
                df = pd.DataFrame()
                # just save a csv with the clustername as file name,so
                # that we later know that an error happened for this cluster.
                df.to_csv(f"{error_path}{cluster_name}.csv")
                print(f"Continue with next cluster.")
                continue
    print("Done investigating diff analysis!")


In [22]:
investigate_differential_analysis_per_tissue(tissues=diff_mb_tissues)

Start analyse cluster for A8WNZ_right_lobe_of_liver:
Start Find specific tf-cos for: A8WNZ_right_lobe_of_liver_c3_Hepatocytes
8
Initial TF-pairs-Count: (351541, 8)
Zero filtered TF-pairs-Count: (351183, 8)
0.21704445727283792
Cluster: A8WNZ_right_lobe_of_liver_c3_Hepatocytes: (0, 8) ,tf-pairs with significant log2fc-changes in comparison to all the other clusters in tissue
Done: Find specific tf co-occurences for clusterA8WNZ_right_lobe_of_liver_c3_Hepatocytes.
Start Find specific tf-cos for: A8WNZ_right_lobe_of_liver_c7_Hepatocytes
8
Initial TF-pairs-Count: (351541, 8)
Zero filtered TF-pairs-Count: (351205, 8)
0.20783400803943555
Cluster: A8WNZ_right_lobe_of_liver_c7_Hepatocytes: (0, 8) ,tf-pairs with significant log2fc-changes in comparison to all the other clusters in tissue
Done: Find specific tf co-occurences for clusterA8WNZ_right_lobe_of_liver_c7_Hepatocytes.
Start Find specific tf-cos for: A8WNZ_right_lobe_of_liver_c8_Adipocyte_progenitor_cells
8
Initial TF-pairs-Count: (351541

Zero filtered TF-pairs-Count: (351106, 13)
0.24051244317882567
Cluster: A8CPH_esophagus_muscularis_mucosa_c1_Fibroblasts: (0, 13) ,tf-pairs with significant log2fc-changes in comparison to all the other clusters in tissue
Done: Find specific tf co-occurences for clusterA8CPH_esophagus_muscularis_mucosa_c1_Fibroblasts.
Start Find specific tf-cos for: A8CPH_esophagus_muscularis_mucosa_c8_Gastric_chief_cells
13
Initial TF-pairs-Count: (351541, 13)
Zero filtered TF-pairs-Count: (351075, 13)
0.22559861535814957
Cluster: A8CPH_esophagus_muscularis_mucosa_c8_Gastric_chief_cells: (0, 13) ,tf-pairs with significant log2fc-changes in comparison to all the other clusters in tissue
Done: Find specific tf co-occurences for clusterA8CPH_esophagus_muscularis_mucosa_c8_Gastric_chief_cells.
Start Find specific tf-cos for: A8CPH_esophagus_muscularis_mucosa_c11_Fibroblasts
13
Initial TF-pairs-Count: (351541, 13)
Zero filtered TF-pairs-Count: (351189, 13)
0.22904083405031456
Cluster: A8CPH_esophagus_muscu

### Answer for: 
a) The found TF-pair occures only in a single cluster of a tissue:
     e.g. Cluster1 has TF1-TF1 and the other cluster in the tissue do not have TF1-TF1.

In [6]:
def find_rows_with_same_value(df:pd.DataFrame) -> pd.DataFrame:
    '''
    Removes all rows of a Dataframe that do not have the same value in each column and
    returns the reduced dataframe.
    '''
    return df[df.apply(lambda x: min(abs(x)) == max(abs(x)), 1)]

In [7]:
#  Get folder names of the tissues.
answers_mb_tissues = get_folder_names_in_folder(rel_folder_path=answers_path)
print(answers_mb_tissues)

['A8WNZ_right_lobe_of_liver', 'ACCQ1_colon_transverse', 'ACCPU_upper_lobe_of_left_lung', 'A8CPH_esophagus_muscularis_mucosa']


In [8]:
'''
We check now the TF-pairs, that have significant log2changes/ 
and a significant cosine value for the investigated clusters.
If the log2foldchange for each contrast (compared clusters in the tissue) has the same value,
the tf-pair is only occureing in the investigated cluster. 
So The found TF-pair occures only in a single cluster of that tissue. 
This tf-pair could be interesting for further investigations.
'''
cluster_with_specials= []
all_cluster_results= []
for tissue in answers_mb_tissues:
    
    print(f"Start analysis {tissue}:")
    answer_specials_save_path = f"{answers_path}{tissue}/specials/"
    # Check if  folder for path already exists for tissue, if not create new one
    if not os.path.exists(answer_specials_save_path):
         pathlib.Path(answer_specials_save_path).mkdir(parents=True, exist_ok=True)
    
    # file names of pd.dataframes with the significant tf-pairs in it.
    files = read_in_file_names_of_folder(rel_path=f"{answers_path}{tissue}")
    for i, file in enumerate(files):
        # print(file)
        cluster_name = file.split('.pkl')[0]
        
        # Read in significant tf-pairs
        df = pd.read_pickle(f"{answers_path}{tissue}/{file}")
        
        all_cluster_results.append(df)

        # has to remove cosine from dataframe, to get only the log2foldchanges
        # for each contrast (compared clusters in the tissue), to check if they have the same value in each column
        df_copy = df.copy()
        cosine_col_name = df_copy.columns[0]
        cosine_col = df_copy[cosine_col_name]
        df_only_log2fc = df_copy.drop(columns=[cosine_col_name])
        
        # find tf-pairs, that have the same value in each log2foldchange column
        df_res = find_rows_with_same_value(df_only_log2fc)
        
        # check if we found a tf-co-occurence which is significant and only occures in one cluster of a tissue
        if df_res.shape[0] > 0: 
            print(f"Cluster: {cluster_name}, wich has tf-pairs that only occure in this cluster for tissue {tissue}.Found: {str(df_res.shape[0])}")
            #add cosine value back to dataframe
            df_res[cosine_col_name]= cosine_col
            cluster_with_specials.append(df_res)
            # save the tf-pairs as csv, if we found some.
            df_res.to_csv(f"{answer_specials_save_path}{cluster_name}.csv")
        
        print(f"Could not find single occurences in cluster {cluster_name}.")
print("DONE")

Start analysis A8WNZ_right_lobe_of_liver:
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c3_Hepatocytes.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c7_Hepatocytes.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c8_Adipocyte_progenitor_cells.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c2_Kupffer_cells.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c6_Hepatocytes.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c1_Adipocytes.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c4_Hepatocytes.
Could not find single occurences in cluster A8WNZ_right_lobe_of_liver_c5_Cholangiocytes.
Start analysis ACCQ1_colon_transverse:
Could not find single occurences in cluster ACCQ1_colon_transverse_c1_Gastric_chief_cells.
Could not find single occurences in cluster ACCQ1_colon_transverse_c3_Enterocytes.
Could not find single occurences in cl

In [9]:
print(f'Number of all investigated clusters: {len(all_cluster_results)}')
cluster_with_high_changes = []
for res in all_cluster_results:
    
    if res.shape[0] > 0:
        cluster_with_high_changes.append(res)
print(f'Number of cluster, that show significant high/ changes for tf-pairs in comparison to the other cluster of theire tissue.')
print(len(cluster_with_high_changes))

Number of all investigated clusters: 31
Number of cluster, that show significant high/ changes for tf-pairs in comparison to the other cluster of theire tissue.
8


In [10]:
print(f'Number of cluster that have tf-pairs that only occure in the investigated cluster in that tissue. (filter thresholds: cosine=0.001 , log2fc=0.10)')
print(len(cluster_with_specials))

Number of cluster that have tf-pairs that only occure in the investigated cluster in that tissue. (filter thresholds: cosine=0.001 , log2fc=0.10)
0


### Checking - The found TF-pair occures only in a single cluster of a tissue

In [26]:
len(cluster_with_specials)
cluster_with_specials[0].T

IndexError: list index out of range